In [1]:
import os
from openai import OpenAI
from pydantic import BaseModel
import json

In [2]:
client = OpenAI(
    api_key = os.getenv('OPEN_AI_API')
)

In [3]:
class rr_flavor(BaseModel):
    rating: int
    reason: str

class rr_freshness(BaseModel):
    rating: int
    reason: str

class rr_healthy(BaseModel):
    rating: int
    reason: str

class rr_service(BaseModel):    
    rating: int
    reason: str

class rr_enviornment(BaseModel):
    rating: int
    reason: str

class rr_value(BaseModel):
    rating: int
    reason: str

class RestaurantEvaluation(BaseModel):
    flavor: rr_flavor
    freshness: rr_freshness
    healthy: rr_healthy
    service: rr_service
    enviornment: rr_enviornment
    value: rr_value

In [4]:
with open('Prompts/Review Summarization Prompts/system_prompt_restaurant_JSON.txt', 'r') as file:
    system_prompt = file.read()

In [5]:
folder_path = 'Datasets/Reviews/Restaurants'

for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            user_prompt = file.read()
            
        #generation into json format
        chat_completion = client.beta.chat.completions.parse(
            model="gpt-4o-mini",
            temperature=0,
            messages=[
                {"role": "system","content": system_prompt},
                {"role": "user","content": user_prompt}
            ],
            response_format=RestaurantEvaluation
        )

        filename_nofiletype = filename[:-4]
        filename_jsontype = filename_nofiletype + '.json'
        result = json.loads(chat_completion.choices[0].message.parsed.json())

        with open('Outputs/Reviews_Summarization/Restaurants/' + filename_jsontype, 'w') as file:
            json.dump(result, file)